In [30]:
import pandas as pd
import numba as np
import sklearn

In [31]:
df = pd.read_csv('C:\\Users\\91772\\ANN\\Churn_Modelling.csv')

In [32]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [33]:
from sklearn import preprocessing

In [34]:
le = preprocessing.LabelEncoder()

In [35]:
df['Gender'] = le.fit_transform(df['Gender'])

In [36]:
from sklearn.preprocessing import OneHotEncoder
OH = OneHotEncoder()
geo_encoder=OH.fit_transform(df[['Geography']]).toarray()
geo_encoder


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [37]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [38]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=OH.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [39]:
data=pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,2,15647311,Hill,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,3,15619304,Onio,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,4,15701354,Boni,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,5,15737888,Mitchell,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
df = df.drop(columns=['CustomerId', 'Surname', 'RowNumber'] , axis=1)

In [11]:
import pickle


In [40]:
with open ('lable_encoder.pkl' , 'wb') as file:
    pickle.dump(le , file)

with open ('one_hot_encoding.pkl' , 'wb')as file:
    pickle.dump(OH , file)    

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [14]:
x= df.drop('Exited', axis=1)
y = df['Exited']

x_train , x_test , y_train , y_test = train_test_split (x , y , random_state= 45 , test_size= 0.2) 

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [15]:
with open ('standardscaler.pkl', 'wb') as file:
    pickle.dump(scaler , file)

In [16]:
df.to_csv('processed_data.csv', index=False)


In [17]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard

In [18]:
model = Sequential([
     Dense(64 , activation= 'relu' , input_shape = (x_train.shape[1],)),
     Dense(34 , activation= 'relu'),
     Dense(1 , activation= 'sigmoid')

]

) 

C:\Users\91772\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 34)             │         2,210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            35 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,077 (12.02 KB)

 Trainable params: 3,077 (12.02 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
import tensorflow as tf

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

loss = tf.keras.losses.BinaryCrossentropy()


In [21]:
model.compile(optimizer=opt , loss= "binary_crossentropy" , metrics= ['accuracy'])

In [22]:
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard()

In [23]:
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [24]:
early_stopping = EarlyStopping(monitor='val_loss' , patience=5 , restore_best_weights=True)

In [25]:
history = model.fit(
    x_train , y_train , validation_data = (x_test,y_test) , epochs = 100,
    callbacks =[tensorflow_callback ,early_stopping ] 
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8099 - loss: 0.4385 - val_accuracy: 0.8465 - val_loss: 0.3727
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8547 - loss: 0.3526 - val_accuracy: 0.8565 - val_loss: 0.3512
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8516 - loss: 0.3525 - val_accuracy: 0.8515 - val_loss: 0.3468
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8569 - loss: 0.3353 - val_accuracy: 0.8570 - val_loss: 0.3377
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8617 - loss: 0.3410 - val_accuracy: 0.8590 - val_loss: 0.3395
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8629 - loss: 0.3260 - val_accuracy: 0.8580 - val_loss: 0.3443
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8660 - loss: 0.3316 - val_accuracy: 0.8610 - val_loss: 0.3367
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8660 - loss: 0.3291 - val_accu

In [26]:
model.save('model.h5')

In [27]:
%load_ext tensorboard



In [28]:
%tensorboard --logdir logs

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.